# Data science project for the 13/04/2019 event
## purpose of the project

Le but de ce projet est de scraper les donnees pour faire une presentation


In [4]:
# importing modules for our work
#importing numpy
import numpy as np
# importing matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
# importing pandas
import pandas as pd
# importing request
import requests
import os, os.path
# importing beautifull soup
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import re
#importing the time module
import time
import datetime

### In the following we will create a text file where we can store our links so that later we can scrap again and do ### comparaison on our data.

# the cell below should be run whenever you want to scrap new links because the intention is to save the link scraped at a particular time in orther to be able to rescrapped the same to analyse over time because if we don't do that each time we will be having different results etc...

In [5]:
# url that will be scrapped
url = 'https://www.jumia.cm/smartphones/'
#pages_to_crawl = 20
#number of pages that will be scrapped
# here we are scrapping 25 pages which represent the number of pages under smartphones
pages_to_crawl = 25
r= requests.get(url)
#soup = BeautifulSoup(r.text, 'html.parser')
''' the for look bellow help get the link of each item and we can open each to investigate more in porder to get insight
from the data store in it'''
#links = soup.find_all(class_="link")
# getting all the links and putting them in an array


# scarping the pages
# here we define a for loop that will loop throught all the pages and save the link into an array
#delete the old file and create a new one
if os.path.exists('links.txt'):
    os.remove("links.txt")
f=open("links.txt","a+")
for p in range(1, pages_to_crawl+1):
    print('Scraping page:', p)
    r = requests.get(url, params={'page' : p})
    soup = BeautifulSoup(r.text, 'html.parser')
    for found  in soup.findAll('a', attrs={'class':'link', 'href': re.compile("^https://")}):
        link1=found.get('href')
        f.write(link1+'\n')
f.close()
    # we pause our code for a second so that we should not be flaged as spam
   # time.sleep(1)
#     for found  in soup.findAll('a', attrs={'class':'link', 'href': re.compile("^https://")}):
#         link1=found.get('href')
#         with open ('links', 'w+') as link:
#             link.write(link1+'\n')
#print (links)


Scraping page: 1
Scraping page: 2
Scraping page: 3
Scraping page: 4
Scraping page: 5
Scraping page: 6
Scraping page: 7
Scraping page: 8
Scraping page: 9
Scraping page: 10
Scraping page: 11
Scraping page: 12
Scraping page: 13
Scraping page: 14
Scraping page: 15
Scraping page: 16
Scraping page: 17
Scraping page: 18
Scraping page: 19
Scraping page: 20
Scraping page: 21
Scraping page: 22
Scraping page: 23
Scraping page: 24
Scraping page: 25


# Things to fix in other to have  a good scrapper

After examining the data scrapped it turned out that there are some data scrapped that the scrapper did not figure it out well. Here are the list of behaviour to fix.
1. number-of-items-sold: the scrapper seemed to performing well here but towards the end we don't have the value meanwhile it was supposed to be 0 if there was nothing so we need to figure out why its not working. I think since the scrapper is taking too long(more than 30 minutes) we can just scrapped the last page in other to tery to understand why
2. same issue with rom and ram but this can be solved by understand and solving the dual appearance of the of the value scrapped
3. fix the camera scrapper too

In [6]:
# difining a dictionarry of data that need to be scrapped
data ={'name':[],
      'seller':[],
      'price':[],
      'number-of-items-sold': [],
      'ram': [],
      'rom': [],
      'screen-size': [],
       'color': [],
       'front-camera':[],
       'rear-camera':[],
       'number_of_sims':[],
       'date': [],
       'os':[]
      }



#next after saving the link to each item, we open each of the and perform the data extraction on them

with open('links.txt') as f:
    for line in f:
        # open the link using the get method
        item_link = requests.get(line.strip())
        item_link_soup =BeautifulSoup(item_link.text, 'html.parser')

        ''' make sure that the seller name exist since this is the brand of the phone , make sure that the brand name is not empty and also that the brand name is different from rock since we have realize that rock, universal is not a phone brand
        and also the type generic is the type of every other item not phones'''
        if item_link_soup.find('div', {'class':'sub-title'}).find('a') is not None and item_link_soup.find('div', {'class':'sub-title'}).find('a').get_text(strip=True).lower() !='generic' and (item_link_soup.find('div', {'class':'sub-title'}).find('a').get_text(strip=True).lower() !='universal') and (item_link_soup.find('div', {'class':'sub-title'}).find('a').get_text(strip=True).lower() !='rock'):
            #getting the seller name after inpection of the link we realize that we could get it from the a tag
            vendeur = item_link_soup.find('div', {'class':'sub-title'}).find('a').get_text(strip=True)
            data['seller'].append(vendeur)
            name = item_link_soup.find('h1', {'class': 'title' }).get_text(strip=True)
            #we realize that in the name of the product, the number of sims cards is indicated but not hence we will check if it specy it if not we put it at 1
            if('dual sim' in name.lower() or 'double puce' in name.lower()):
                data['number_of_sims'].append(2)
            else:
                data['number_of_sims'].append(1)
            data['name'].append(name)
            #getting the price of items
            prise = item_link_soup.find('span', {'dir':'ltr'})
            prize = prise.get_text(strip=True).replace(',', '')
            data['price'].append(float(prize))

            # getting the numbers of items sold
            sold = item_link_soup.find('div', {'class':'-ptxxs'})
            if sold  is not None :
                if sold.find('span', {'class': 'text color-default bold -prxs -inline-block -bold'})is not None:
                    vendu = sold.find('span', {'class': 'text color-default bold -prxs -inline-block -bold'}).get_text(strip=True)


                # on the website the number of items sold are presents in form of X+ or <x so we add or substract one to reflect it
                    if vendu.endswith('+'):
                        vendu1 = vendu.replace('+', '')
                        vendu2 = vendu1.replace(',', '')
                        vendu_final = float(vendu2) + 1
                        data['number-of-items-sold'].append(vendu_final)
                    # here due to the fact that jumia.cm doesn't give the exact number sold, we add 1 if its said to be more than or sousract if to be less than.
                    elif vendu.startswith('<'):
                        vendu1 = vendu.replace('<', '')
                        vendu2 = vendu1.replace(',', '')
                        vendu_final = float(vendu2) - 1
                        data['number-of-items-sold'].append(vendu_final)
                    else :
                        data['number-of-items-sold'].append(0)
            # this else can be removed was having an error in my notebook and thought it was the problem but actually no
            else:
                ok = 'not know'

            #getting the color of the phone and various other 
            features = []

            for caracteristiques in item_link_soup.find_all('div', {'class':'osh-row '}):
                value = caracteristiques.find_all('div', {'class': 'osh-col'})
                features.append(value)


            # we loop over the features and get the necessary info
            #print(features)
            for i in range(len(features)):
                    #print(value1)
                    # we unpack the value of the array
                    try:
                        x, y = features[i]
                        if 'couleur principale' in x.get_text(strip=True).lower():
                            #couleur = value[1].get_text(strip=True).lower()
                            #print(couleur)
                            data['color'].append(y.get_text(strip=True).lower())
                            #print(value)
                        elif 'taille de' in x.get_text(strip=True).lower():
                             data['screen-size'].append(y.get_text(strip=True).lower())
                        elif 'ram syst' in x.get_text(strip=True).lower():
                            ram2 = float(y.get_text(strip=True).lower())
                            # if the value of the ram is 512 it means that its 512mb and we convert it accordingly
                            if ram2 ==512:
                                data['ram'].append(0.512)
                            else:
                                data['ram'].append(float(y.get_text(strip=True).lower()))
                        elif 'capacit' in x.get_text(strip=True).lower():
                             data['rom'].append(float(y.get_text(strip=True).lower()))
                        elif 'os' in x.get_text(strip=True).lower():
                             data['os'].append(y.get_text(strip=True).lower())
                        else:
                            a = 'ok'
                    except ValueError:
                        pass
            '''getting the camera info the site does not specify it in a simple way that we can just scrap so we are getting the
            the camera info by using the regex''' 
            if(len(item_link_soup.find_all('ul')[4].find_all('li'))>2):
                cam1 = str(item_link_soup.find_all('ul')[4].find_all('li')[2].get_text(strip= True)).lower()
                if('caméra' in cam1 or 'photo' in cam1):
                    arr1 = [float(s) for s in re.findall(r'-?\d+\.?\d*', cam1)]
                    arr2 = [x for x in arr1 if x != 0.0 and x<26]
                    if len(arr2) == 2 and arr2[1]<25:
                        data['rear-camera'].append(arr2[0])
                        data['front-camera'].append(arr2[1])
                    elif len(arr2) == 2 and arr2[1]>25:
                        data['rear-camera'].append(arr2[0])
                        data['front-camera'].append(0)
                    elif len(arr2)==1 :
                        data['rear-camera'].append(arr2[0])
                        data['front-camera'].append(0)
                    elif len(arr2)==3 and arr2[1]<26 and arr2[2]<26:
                        data['rear-camera'].append(arr2[0]+arr2[1])
                        data['front-camera'].append(2)
                    elif len(arr2)>3:
                        data['rear-camera'].append(arr2[0])
                        data['front-camera'].append(0)
                    elif len(arr2)==4:
                        data['rear-camera'].append(arr2[0]+arr2[1])
                        data['front-camera'].append(arr2[2]+arr2(3))
                    else:
                        arr3 = 0
            #data['vendeur'].append('unknow')
            # we are using sleep in other for our scrapper to waith for one second before going again
            #time.sleep(1)
            scrap_time =datetime.datetime.now()
            data['date'].append(scrap_time.strftime("%c"))
            # at the end the scrapper wait for 0.1 second before startint with the next link
            time.sleep(0.100)

''' We are having thinks twice so that s why we are dividing by two.'''
        
# casting colors in other to get the real color
'''couleur_to =list( data['color'])
couleur_ok = []
ram_ok = []
rom_ok = []
taille_ok = []
#print(couleur_to)
ram_to = list(data['ram'])
print(ram_to)
rom_to = list(data['rom'])
taille_to = list(data['screen-size'])
print(int(len(couleur_to)/2))
for i in range(int(len(taille_to)/2)):
    taille_ok.append(taille_to[i*2])
    data['screen-size'] = taille_ok
for i in range(int(len(ram_to)/2)):
    ram_ok.append(ram_to[i*2])
    data['ram'] = ram_ok
for i in range(int(len(rom_to)/2)):
    rom_ok.append(rom_to[i*2])
    data['rom'] = rom_ok
for i in range(int(len(couleur_to)/2)):
    
    #del couleur_to[i*2]
    couleur_ok.append(couleur_to[i*2])
    #ram_ok.append(ram_to[i*2])
    #rom_ok.append(rom_to[i*2])
    #taille_ok.append(taille_to[i*2])
    data['color'] = couleur_ok '''

#print(data)

#converting the dictionnary to a dataframe
frame = pd.concat([pd.Series(v, name=k) for k, v in data.items()], axis=1)
fileName = fileName = 'jumia_cm_smartphones_scraped_on_' + datetime.datetime.now().strftime("%c").replace(' ', '_')+('.csv')
#print(frame)
# here we export the data to csv file
export_csv = frame.to_csv (fileName, index = False)

# analysing the csv file save


In [8]:
# reading the saved file
df = pd.read_csv('jumia_cm_smartphones_scraped_on_Sun_May_26_21:47:08_2019.csv')
print(df.tail())
print(df.info())

                                                  name   seller     price  \
905  Galaxy Note 8 - 64 Go HDD - 6 Go RAM - 6,3'' -...  Samsung  275000.0   
906  Galaxy S7 Edge 64Go HDD - 4Go RAM - Or - Poche...  Samsung  149500.0   
907  Galaxy S7 - 32 Go HDD - 4 Go RAM - 5,1'' - Or ...  Samsung  129500.0   
908                Chargeur Pour Samsung Galaxy - Noir  Samsung    5000.0   
909  Galaxy S9 Plus - 64Go HDD - 6Go RAM - 6,2" - N...  Samsung  390000.0   

     number-of-items-sold  ram  rom  screen-size color  front-camera  \
905                   NaN  NaN  NaN          NaN  noir           NaN   
906                   NaN  NaN  NaN          NaN    or           NaN   
907                   NaN  NaN  NaN          NaN    or           NaN   
908                   NaN  NaN  NaN          NaN  noir           NaN   
909                   NaN  NaN  NaN          NaN  noir           NaN   

     rear-camera  number_of_sims                      date   os  
905          NaN               1  Sun 